## Neural Network Architecture

In [10]:
model

Multi_AlexnetMap_v3(
  (rgb_features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (d_features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (features): Sequential(
    (0): Conv2d(128, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.3, inplace=False)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU(inplace=True)
    (6): Dropout(p=0.3, inplace=False)
    (7): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): Dropout(p=0.3, inplace=False)


## Regular Testing for Non-Shuffled Data

In [1]:
"""
This file tests the model's performance on the testing dataset.

For CLS, this script returns the testing accuracy.
For Grasp, this script returns the testing accuracy and visualizes the
grasp prediction.


Comment or uncomment certain lines of code for swapping between
training CLS model and Grasping model.

E.g. Uncomment the lines with NO SPACE between '#' and the codes: 
# Get test acc for CLS model
#accuracy, loss = get_test_acc(model)
# Get test acc for Grasp model
accuracy, loss = get_grasp_acc(model)

----->

# Get test acc for CLS model
accuracy, loss = get_test_acc(model)
# Get test acc for Grasp model
#accuracy, loss = get_grasp_acc(model)
"""

import torch
import os
import time
import sys
from utils.parameters import Params
from multi_task_models.grcn_multi_alex import Multi_AlexnetMap_v3
from training.single_task.evaluation import get_cls_acc, get_grasp_acc, visualize_grasp, visualize_cls

params = Params()

model_name = params.MODEL_NAME
weights_dir = params.MODEL_PATH
weights_path = os.path.join(weights_dir, model_name, model_name + '_final.pth')
# AlexNet with 1st, 2nd layer pretrained on Imagenet
model =  Multi_AlexnetMap_v3().to('cuda')
model.load_state_dict(torch.load(weights_path))
model.eval()
path = params.TEST_PATH
# if len(sys.argv) > 1:
#     path = params.TEST_PATH_SHUFFLE

# Visualize CLS predictions one by one
#visualize_cls(model)
# Visualize grasp predictions one by one
#visualize_grasp(model)


/home/billy/anaconda3/envs/gr_cls_multitask/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/billy/anaconda3/envs/gr_cls_multitask/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/tmp/ipykernel_50107/1692255469.py:41: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/py

In [2]:
def register_custom_hooks(model):
    """
    Registers hooks on specific layers of the model.
    Returns a dictionary to store activations and a list of hooks.
    """
    layers_to_track = [
        ('rgb_features', 0),
        ('features', 0),
        ('features', 4),
        ('features', 7),
        ('features', 10)
    ]

    activations = {f"{layer_name}_{layer_idx}": [] for layer_name, layer_idx in layers_to_track}

    def hook_fn(layer_name):
        def hook(module, input, output):
            activations[layer_name].append(output.clone().detach())
        return hook

    hooks = []
    for layer_name, layer_idx in layers_to_track:
        if layer_name in ['rgb_features', 'd_features']:
            block = getattr(model, layer_name)
        else:
            block = getattr(model, layer_name)
        hook_key = f"{layer_name}_{layer_idx}"
        hooks.append(block[layer_idx].register_forward_hook(hook_fn(hook_key)))

    return hooks, activations

In [3]:
# Register hooks
hooks, activations = register_custom_hooks(model)

In [4]:
# Get test acc for CLS model
c_accuracy, c_loss = get_cls_acc(model, include_depth=True, seed=None, dataset=path, truncation=None)

Dataset size: 400
Training steps: 360 -- Val steps: 40


In [5]:
for layer_name, acts in activations.items():
    if len(acts) > 1:
        acts = torch.cat(acts, dim=0)  # Shape: [Batch x Channels x Height x Width]
        avg_activation = acts.mean(dim=(0, 2, 3))  # Shape: [Channels]
        top_indices = torch.topk(avg_activation, 10).indices
        print(f"Top 10 feature maps for {layer_name}: {top_indices.tolist()}")

Top 10 feature maps for rgb_features_0: [47, 5, 17, 62, 10, 25, 14, 33, 37, 2]
Top 10 feature maps for features_0: [19, 2, 7, 9, 21, 20, 13, 11, 5, 6]
Top 10 feature maps for features_4: [7, 50, 22, 16, 56, 27, 62, 44, 17, 59]
Top 10 feature maps for features_7: [11, 47, 37, 57, 62, 29, 60, 39, 52, 6]
Top 10 feature maps for features_10: [24, 18, 30, 45, 36, 9, 38, 54, 15, 16]


In [6]:
acts.shape

torch.Size([400, 64, 13, 13])

In [7]:
# Debugging: Check stored activations
for layer_name, acts in activations.items():
    print(f"Layer: {layer_name}, Activations Count: {len(acts)}")
    if len(acts) > 0:
        print(f"Activation Shape: {acts[0].shape}")

Layer: rgb_features_0, Activations Count: 400
Activation Shape: torch.Size([1, 64, 55, 55])
Layer: features_0, Activations Count: 400
Activation Shape: torch.Size([1, 32, 27, 27])
Layer: features_4, Activations Count: 400
Activation Shape: torch.Size([1, 64, 13, 13])
Layer: features_7, Activations Count: 400
Activation Shape: torch.Size([1, 64, 13, 13])
Layer: features_10, Activations Count: 400
Activation Shape: torch.Size([1, 64, 13, 13])


In [8]:
# Get test acc for Grasp model
accuracy, loss = get_grasp_acc(model, include_depth=True, seed=None, dataset=path, truncation=None)



Dataset size: 400
Training steps: 72 -- Val steps: 8


In [9]:
# Debugging: Check stored activations
for layer_name, acts in activations.items():
    print(f"Layer: {layer_name}, Activations Count: {len(acts)}")
    if len(acts) > 0:
        print(f"Activation Shape: {acts[0].shape}")

Layer: rgb_features_0, Activations Count: 480
Activation Shape: torch.Size([1, 64, 55, 55])
Layer: features_0, Activations Count: 480
Activation Shape: torch.Size([1, 32, 27, 27])
Layer: features_4, Activations Count: 480
Activation Shape: torch.Size([1, 64, 13, 13])
Layer: features_7, Activations Count: 480
Activation Shape: torch.Size([1, 64, 13, 13])
Layer: features_10, Activations Count: 480
Activation Shape: torch.Size([1, 64, 13, 13])


In [ ]:
# Debugging: Check stored activations
for layer_name, acts in activations.items():
    print(f"Layer: {layer_name}, Activations Count: {len(acts)}")
    if len(acts) > 0:
        print(f"Activation Shape: {acts[0].shape}")

Layer: rgb_features_0, Activations Count: 80
Activation Shape: torch.Size([5, 64, 55, 55])
Layer: features_0, Activations Count: 80
Activation Shape: torch.Size([5, 32, 27, 27])
Layer: features_4, Activations Count: 80
Activation Shape: torch.Size([5, 64, 13, 13])
Layer: features_7, Activations Count: 80
Activation Shape: torch.Size([5, 64, 13, 13])
Layer: features_10, Activations Count: 80
Activation Shape: torch.Size([5, 64, 13, 13])


In [10]:
for layer_name, acts in activations.items():
    if len(acts) > 1:
        acts = torch.cat(acts, dim=0)  # Shape: [Batch x Channels x Height x Width]
        avg_activation = acts.mean(dim=(0, 2, 3))  # Shape: [Channels]
        top_indices = torch.topk(avg_activation, 10).indices
        print(f"Top 10 feature maps for {layer_name}: {top_indices.tolist()}")

Top 10 feature maps for rgb_features_0: [47, 5, 17, 62, 10, 25, 14, 33, 37, 2]
Top 10 feature maps for features_0: [19, 2, 7, 9, 21, 20, 13, 11, 5, 6]
Top 10 feature maps for features_4: [7, 50, 22, 16, 56, 27, 62, 44, 17, 59]
Top 10 feature maps for features_7: [11, 47, 37, 57, 62, 29, 60, 39, 52, 6]
Top 10 feature maps for features_10: [24, 18, 30, 45, 36, 9, 38, 54, 15, 16]


In [ ]:
for layer_name, acts in activations.items():
    if len(acts) > 1:
        acts = torch.cat(acts, dim=0)  # Shape: [Batch x Channels x Height x Width]
        avg_activation = acts.mean(dim=(0, 2, 3))  # Shape: [Channels]
        top_indices = torch.topk(avg_activation, 10).indices
        print(f"Top 10 feature maps for {layer_name}: {top_indices.tolist()}")

Top 10 feature maps for rgb_features_0: [47, 5, 17, 62, 10, 25, 14, 33, 37, 2]
Top 10 feature maps for features_0: [19, 2, 7, 9, 21, 20, 13, 11, 5, 6]
Top 10 feature maps for features_4: [7, 50, 22, 16, 56, 27, 62, 44, 17, 59]
Top 10 feature maps for features_7: [11, 47, 37, 57, 62, 29, 60, 39, 52, 6]
Top 10 feature maps for features_10: [24, 18, 30, 45, 36, 9, 38, 54, 15, 16]


In [11]:
print('Grasp: %s' % accuracy, loss)
print('CLS: %s' % c_accuracy, c_loss)

Grasp: 81.25 0.0
CLS: 85.0 0.0


## For Shuffled Data

### Combined Data

In [1]:
"""
This file tests the model's performance on the testing dataset.

For CLS, this script returns the testing accuracy.
For Grasp, this script returns the testing accuracy and visualizes the
grasp prediction.


Comment or uncomment certain lines of code for swapping between
training CLS model and Grasping model.

E.g. Uncomment the lines with NO SPACE between '#' and the codes: 
# Get test acc for CLS model
#accuracy, loss = get_test_acc(model)
# Get test acc for Grasp model
accuracy, loss = get_grasp_acc(model)

----->

# Get test acc for CLS model
accuracy, loss = get_test_acc(model)
# Get test acc for Grasp model
#accuracy, loss = get_grasp_acc(model)
"""

import torch
import os
import time
import sys
from utils.parameters import Params
from multi_task_models.grcn_multi_alex import Multi_AlexnetMap_v3
from training.single_task.evaluation import get_cls_acc, get_grasp_acc, visualize_grasp, visualize_cls

params = Params()

model_name = params.MODEL_NAME
weights_dir = params.MODEL_PATH
weights_path = os.path.join(weights_dir, model_name, model_name + '_final.pth')
# AlexNet with 1st, 2nd layer pretrained on Imagenet
model =  Multi_AlexnetMap_v3().to('cuda')
model.load_state_dict(torch.load(weights_path))
model.eval()
path = params.TEST_PATH_SHUFFLE
# if len(sys.argv) > 1:
#     path = params.TEST_PATH_SHUFFLE

# Visualize CLS predictions one by one
#visualize_cls(model)
# Visualize grasp predictions one by one
#visualize_grasp(model)

/home/billy/anaconda3/envs/gr_cls_multitask/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/billy/anaconda3/envs/gr_cls_multitask/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/tmp/ipykernel_99904/1716835784.py:41: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/py

In [2]:
def register_custom_hooks(model):
    """
    Registers hooks on specific layers of the model.
    Returns a dictionary to store activations and a list of hooks.
    """
    layers_to_track = [
        ('rgb_features', 0),
        ('features', 0),
        ('features', 4),
        ('features', 7),
        ('features', 10)
    ]

    activations = {f"{layer_name}_{layer_idx}": [] for layer_name, layer_idx in layers_to_track}

    def hook_fn(layer_name):
        def hook(module, input, output):
            activations[layer_name].append(output.clone().detach())
        return hook

    hooks = []
    for layer_name, layer_idx in layers_to_track:
        if layer_name in ['rgb_features', 'd_features']:
            block = getattr(model, layer_name)
        else:
            block = getattr(model, layer_name)
        hook_key = f"{layer_name}_{layer_idx}"
        hooks.append(block[layer_idx].register_forward_hook(hook_fn(hook_key)))

    return hooks, activations

In [3]:
# Register hooks
hooks, activations = register_custom_hooks(model)

In [4]:
# Get test acc for CLS model
c_accuracy, c_loss = get_cls_acc(model, include_depth=True, seed=None, dataset=path, truncation=None)

Dataset size: 1200
Training steps: 216 -- Val steps: 24


In [5]:
# Debugging: Check stored activations
for layer_name, acts in activations.items():
    print(f"Layer: {layer_name}, Activations Count: {len(acts)}")
    if len(acts) > 0:
        print(f"Activation Shape: {acts[0].shape}")

Layer: rgb_features_0, Activations Count: 240
Activation Shape: torch.Size([5, 64, 55, 55])
Layer: features_0, Activations Count: 240
Activation Shape: torch.Size([5, 32, 27, 27])
Layer: features_4, Activations Count: 240
Activation Shape: torch.Size([5, 64, 13, 13])
Layer: features_7, Activations Count: 240
Activation Shape: torch.Size([5, 64, 13, 13])
Layer: features_10, Activations Count: 240
Activation Shape: torch.Size([5, 64, 13, 13])


In [6]:
for layer_name, acts in activations.items():
    if len(acts) > 1:
        acts = torch.cat(acts, dim=0)  # Shape: [Batch x Channels x Height x Width]
        avg_activation = acts.mean(dim=(0, 2, 3))  # Shape: [Channels]
        top_indices = torch.topk(avg_activation, 10).indices
        print(f"Top 10 feature maps for {layer_name}: {top_indices.tolist()}")

Top 10 feature maps for rgb_features_0: [47, 5, 17, 62, 10, 25, 14, 33, 37, 2]
Top 10 feature maps for features_0: [19, 2, 9, 7, 21, 13, 20, 5, 11, 10]
Top 10 feature maps for features_4: [50, 16, 7, 22, 56, 62, 44, 27, 59, 17]
Top 10 feature maps for features_7: [11, 57, 37, 47, 62, 52, 24, 0, 39, 55]
Top 10 feature maps for features_10: [45, 24, 18, 30, 36, 38, 9, 8, 22, 54]


In [7]:
# Get test acc for Grasp model
accuracy, loss = get_grasp_acc(model, include_depth=True, seed=None, dataset=path, truncation=None)

Dataset size: 1200
Training steps: 1080 -- Val steps: 120
index 4 is out of bounds for dimension 2 with size 4
index 4 is out of bounds for dimension 2 with size 4
index 4 is out of bounds for dimension 2 with size 4
index 4 is out of bounds for dimension 2 with size 4
index 4 is out of bounds for dimension 2 with size 4
index 4 is out of bounds for dimension 2 with size 4
index 4 is out of bounds for dimension 2 with size 4
index 4 is out of bounds for dimension 2 with size 4
index 4 is out of bounds for dimension 2 with size 4
index 4 is out of bounds for dimension 2 with size 4
index 4 is out of bounds for dimension 2 with size 4
index 4 is out of bounds for dimension 2 with size 4
index 4 is out of bounds for dimension 2 with size 4
index 4 is out of bounds for dimension 2 with size 4
index 4 is out of bounds for dimension 2 with size 4
index 4 is out of bounds for dimension 2 with size 4
index 4 is out of bounds for dimension 2 with size 4
index 4 is out of bounds for dimension 2 

In [6]:
# Separately
for layer_name, acts in activations.items():
    if len(acts) > 1:
        acts = torch.cat(acts, dim=0)  # Shape: [Batch x Channels x Height x Width]
        avg_activation = acts.mean(dim=(0, 2, 3))  # Shape: [Channels]
        top_indices = torch.topk(avg_activation, 10).indices
        print(f"Top 10 feature maps for {layer_name}: {top_indices.tolist()}")

Top 10 feature maps for rgb_features_0: [47, 5, 17, 62, 10, 25, 14, 33, 37, 2]
Top 10 feature maps for features_0: [19, 2, 9, 7, 21, 13, 20, 5, 11, 10]
Top 10 feature maps for features_4: [50, 16, 7, 22, 56, 62, 44, 27, 59, 17]
Top 10 feature maps for features_7: [11, 57, 37, 47, 62, 52, 24, 0, 39, 55]
Top 10 feature maps for features_10: [45, 24, 18, 30, 36, 38, 9, 8, 22, 54]


In [ ]:
# Consecutively
for layer_name, acts in activations.items():
    if len(acts) > 1:
        acts = torch.cat(acts, dim=0)  # Shape: [Batch x Channels x Height x Width]
        avg_activation = acts.mean(dim=(0, 2, 3))  # Shape: [Channels]
        top_indices = torch.topk(avg_activation, 10).indices
        print(f"Top 10 feature maps for {layer_name}: {top_indices.tolist()}")

Top 10 feature maps for rgb_features_0: [47, 5, 17, 62, 10, 25, 14, 33, 37, 2]
Top 10 feature maps for features_0: [19, 2, 9, 7, 21, 13, 20, 5, 11, 10]
Top 10 feature maps for features_4: [50, 16, 7, 22, 56, 62, 44, 27, 59, 17]
Top 10 feature maps for features_7: [11, 57, 37, 47, 62, 52, 24, 0, 39, 55]
Top 10 feature maps for features_10: [45, 24, 18, 30, 36, 38, 9, 8, 22, 54]


In [7]:
# Separately
# Debugging: Check stored activations
for layer_name, acts in activations.items():
    print(f"Layer: {layer_name}, Activations Count: {len(acts)}")
    if len(acts) > 0:
        print(f"Activation Shape: {acts[0].shape}")

Layer: rgb_features_0, Activations Count: 1200
Activation Shape: torch.Size([1, 64, 55, 55])
Layer: features_0, Activations Count: 1200
Activation Shape: torch.Size([1, 32, 27, 27])
Layer: features_4, Activations Count: 1200
Activation Shape: torch.Size([1, 64, 13, 13])
Layer: features_7, Activations Count: 1200
Activation Shape: torch.Size([1, 64, 13, 13])
Layer: features_10, Activations Count: 1200
Activation Shape: torch.Size([1, 64, 13, 13])


In [8]:
# Consecutively
# Debugging: Check stored activations
for layer_name, acts in activations.items():
    print(f"Layer: {layer_name}, Activations Count: {len(acts)}")
    if len(acts) > 0:
        print(f"Activation Shape: {acts[0].shape}")

Layer: rgb_features_0, Activations Count: 1440
Activation Shape: torch.Size([5, 64, 55, 55])
Layer: features_0, Activations Count: 1440
Activation Shape: torch.Size([5, 32, 27, 27])
Layer: features_4, Activations Count: 1440
Activation Shape: torch.Size([5, 64, 13, 13])
Layer: features_7, Activations Count: 1440
Activation Shape: torch.Size([5, 64, 13, 13])
Layer: features_10, Activations Count: 1440
Activation Shape: torch.Size([5, 64, 13, 13])


### One Horizontal Split Keeps Below

In [1]:
"""
This file tests the model's performance on the testing dataset.

For CLS, this script returns the testing accuracy.
For Grasp, this script returns the testing accuracy and visualizes the
grasp prediction.


Comment or uncomment certain lines of code for swapping between
training CLS model and Grasping model.

E.g. Uncomment the lines with NO SPACE between '#' and the codes: 
# Get test acc for CLS model
#accuracy, loss = get_test_acc(model)
# Get test acc for Grasp model
accuracy, loss = get_grasp_acc(model)

----->

# Get test acc for CLS model
accuracy, loss = get_test_acc(model)
# Get test acc for Grasp model
#accuracy, loss = get_grasp_acc(model)
"""

import torch
import os
import time
import sys
from utils.parameters import Params
from multi_task_models.grcn_multi_alex import Multi_AlexnetMap_v3
from training.single_task.evaluation import get_cls_acc, get_grasp_acc, visualize_grasp, visualize_cls

params = Params()

model_name = params.MODEL_NAME
weights_dir = params.MODEL_PATH
weights_path = os.path.join(weights_dir, model_name, model_name + '_final.pth')
# AlexNet with 1st, 2nd layer pretrained on Imagenet
model =  Multi_AlexnetMap_v3().to('cuda')
model.load_state_dict(torch.load(weights_path))
model.eval()
path = params.TEST_PATH_SHUFFLE
# if len(sys.argv) > 1:
#     path = params.TEST_PATH_SHUFFLE

# Visualize CLS predictions one by one
#visualize_cls(model)
# Visualize grasp predictions one by one
#visualize_grasp(model)


/home/billy/anaconda3/envs/gr_cls_multitask/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/billy/anaconda3/envs/gr_cls_multitask/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/tmp/ipykernel_77045/609354409.py:41: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pyt

In [2]:
def register_custom_hooks(model):
    """
    Registers hooks on specific layers of the model.
    Returns a dictionary to store activations and a list of hooks.
    """
    layers_to_track = [
        ('rgb_features', 0),
        ('features', 0),
        ('features', 4),
        ('features', 7),
        ('features', 10)
    ]

    activations = {f"{layer_name}_{layer_idx}": [] for layer_name, layer_idx in layers_to_track}

    def hook_fn(layer_name):
        def hook(module, input, output):
            activations[layer_name].append(output.clone().detach())
        return hook

    hooks = []
    for layer_name, layer_idx in layers_to_track:
        if layer_name in ['rgb_features', 'd_features']:
            block = getattr(model, layer_name)
        else:
            block = getattr(model, layer_name)
        hook_key = f"{layer_name}_{layer_idx}"
        hooks.append(block[layer_idx].register_forward_hook(hook_fn(hook_key)))

    return hooks, activations

In [3]:
# Register hooks
hooks, activations = register_custom_hooks(model)

In [ ]:
# Get test acc for CLS model
c_accuracy, c_loss = get_cls_acc(model, include_depth=True, seed=None, dataset=path, truncation=None)

In [17]:
for layer_name, acts in activations.items():
    if len(acts) > 1:
        acts = torch.cat(acts, dim=0)  # Shape: [Batch x Channels x Height x Width]
        avg_activation = acts.mean(dim=(0, 2, 3))  # Shape: [Channels]
        top_indices = torch.topk(avg_activation, 10).indices
        print(f"Top 10 feature maps for {layer_name}: {top_indices.tolist()}")

Top 10 feature maps for rgb_features: [47, 5, 43, 61, 46, 62, 17, 37, 38, 10]
Top 10 feature maps for d_features: [8, 53, 62, 40, 47, 5, 17, 32, 43, 10]
Top 10 feature maps for features: [24, 18, 38, 9, 36, 45, 54, 15, 30, 60]
Top 10 feature maps for cls: [15, 8, 12, 5, 0, 9, 1, 11, 10, 3]


In [ ]:
## This is the output for 1 Split vertical Keeps Random

Top 10 feature maps for rgb_features: [47, 5, 43, 61, 46, 62, 17, 37, 38, 10]
Top 10 feature maps for d_features: [8, 53, 40, 62, 47, 5, 17, 32, 43, 23]
Top 10 feature maps for features: [24, 18, 38, 9, 36, 45, 54, 30, 60, 15]
Top 10 feature maps for cls: [15, 8, 1, 12, 5, 9, 3, 0, 11, 14]


In [ ]:
## This is the output for 1 Split Horizontal Keeps Random

Top 10 feature maps for rgb_features: [47, 5, 43, 61, 46, 62, 17, 37, 38, 10]
Top 10 feature maps for d_features: [8, 53, 40, 62, 47, 5, 17, 32, 43, 23]
Top 10 feature maps for features: [24, 18, 38, 9, 36, 45, 54, 30, 15, 60]
Top 10 feature maps for cls: [8, 15, 12, 1, 5, 9, 0, 3, 11, 14]


In [ ]:
## This is the output for 1 Split Horizontal Keeps Above

Top 10 feature maps for rgb_features: [47, 5, 43, 61, 46, 62, 17, 37, 38, 10]
Top 10 feature maps for d_features: [8, 53, 40, 62, 47, 5, 17, 32, 43, 57]
Top 10 feature maps for features: [24, 18, 38, 9, 36, 45, 30, 15, 54, 60]
Top 10 feature maps for cls: [8, 15, 12, 1, 5, 9, 3, 11, 0, 14]


In [ ]:
## This is the output for 1 Split Horizontal Keeps Below

Top 10 feature maps for rgb_features: [47, 5, 43, 61, 46, 62, 17, 37, 38, 10]
Top 10 feature maps for d_features: [8, 53, 40, 62, 47, 5, 17, 32, 43, 23]
Top 10 feature maps for features: [24, 18, 38, 36, 9, 54, 45, 30, 60, 15]
Top 10 feature maps for cls: [8, 12, 15, 1, 5, 9, 3, 11, 0, 14]


In [4]:
# Get test acc for Grasp model
accuracy, loss = get_grasp_acc(model, include_depth=True, seed=None, dataset=path, truncation=None)

Dataset size: 400
Training steps: 360 -- Val steps: 40


index 4 is out of bounds for dimension 2 with size 4
index 4 is out of bounds for dimension 2 with size 4
index 4 is out of bounds for dimension 2 with size 4
index 4 is out of bounds for dimension 2 with size 4
index 4 is out of bounds for dimension 2 with size 4
index 4 is out of bounds for dimension 2 with size 4
index 4 is out of bounds for dimension 2 with size 4
index 4 is out of bounds for dimension 2 with size 4
index 4 is out of bounds for dimension 2 with size 4
index 4 is out of bounds for dimension 2 with size 4
index 4 is out of bounds for dimension 2 with size 4
index 4 is out of bounds for dimension 2 with size 4
index 4 is out of bounds for dimension 2 with size 4
index 4 is out of bounds for dimension 2 with size 4
index 4 is out of bounds for dimension 2 with size 4
index 4 is out of bounds for dimension 2 with size 4
index 4 is out of bounds for dimension 2 with size 4
index 4 is out of bounds for dimension 2 with size 4
index 4 is out of bounds for dimension 2 with 

In [ ]:
for layer_name, acts in activations.items():
    if len(acts) > 1:
        acts = torch.cat(acts, dim=0)  # Shape: [Batch x Channels x Height x Width]
        avg_activation = acts.mean(dim=(0, 2, 3))  # Shape: [Channels]
        top_indices = torch.topk(avg_activation, 10).indices
        print(f"Top 10 feature maps for {layer_name}: {top_indices.tolist()}")

## This is the output for 1 Split Horizontal + 1 Split Vertical, both keep random

Top 10 feature maps for rgb_features: [47, 5, 43, 61, 46, 62, 17, 37, 38, 10]
Top 10 feature maps for d_features: [8, 53, 62, 40, 47, 5, 17, 32, 43, 10]
Top 10 feature maps for features: [24, 18, 38, 9, 36, 45, 54, 15, 30, 60]
Top 10 feature maps for grasp: [2, 8, 3, 7, 10, 5, 4, 12, 1, 11]


In [ ]:
## This is the output for 1 Split vertical Keeps random

Top 10 feature maps for rgb_features: [47, 5, 43, 61, 46, 62, 17, 37, 38, 10]
Top 10 feature maps for d_features: [8, 53, 40, 62, 47, 5, 17, 32, 43, 23]
Top 10 feature maps for features: [24, 18, 38, 9, 36, 45, 54, 30, 60, 15]
Top 10 feature maps for grasp: [2, 3, 7, 8, 10, 5, 12, 4, 1, 11]


In [ ]:
## This is the output for 1 Split Horizontal Keeps random

Top 10 feature maps for rgb_features: [47, 5, 43, 61, 46, 62, 17, 37, 38, 10]
Top 10 feature maps for d_features: [8, 53, 40, 62, 47, 5, 17, 32, 43, 23]
Top 10 feature maps for features: [24, 18, 38, 9, 36, 45, 54, 30, 15, 60]
Top 10 feature maps for grasp: [2, 3, 7, 8, 10, 5, 4, 12, 1, 11]


In [ ]:
## This is the output for 1 Split Horizontal Keeps Above

Top 10 feature maps for rgb_features: [47, 5, 43, 61, 46, 62, 17, 37, 38, 10]
Top 10 feature maps for d_features: [8, 53, 40, 62, 47, 5, 17, 32, 43, 57]
Top 10 feature maps for features: [24, 18, 38, 9, 36, 45, 30, 15, 54, 60]
Top 10 feature maps for grasp: [2, 3, 7, 8, 10, 5, 12, 4, 1, 11]


In [36]:
## This is the output for 1 Split Horizontal Keeps Below

Top 10 feature maps for rgb_features: [47, 5, 43, 61, 46, 62, 17, 37, 38, 10]
Top 10 feature maps for d_features: [8, 53, 40, 62, 47, 5, 17, 32, 43, 23]
Top 10 feature maps for features: [24, 18, 38, 36, 9, 54, 45, 30, 60, 15]
Top 10 feature maps for grasp: [2, 3, 7, 8, 10, 5, 12, 4, 1, 11]


In [ ]:
## This is the output for 1 Split Horizontal Keeps random

Grasp: 29.25 0.0
CLS: 48.25 0.0


In [ ]:
## This is the output for 1 Split Horizontal Keeps Above

Grasp: 40.0 0.0
CLS: 45.0 0.0


In [ ]:
## This is the output for 1 Split Horizontal Keeps Below

Grasp: 26.0 0.0
CLS: 52.5 0.0
